In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#import and setup training dataset
TRAINING_DATASET_PATH = "./Fashion_datasets/myntradataset/"
training_data = pd.read_csv(TRAINING_DATASET_PATH + "styles.csv", error_bad_lines=False)
training_data['image'] = training_data.apply(lambda row: str(row['id']) + ".jpg", axis=1)
training_data = training_data.sample(frac=1).reset_index(drop=True)

#import and setup test dataset
TEST_DATASET_PATH = "./Fashion_datasets/"
test_data = pd.read_csv(TEST_DATASET_PATH + "styles.csv", nrows=128, error_bad_lines=False)
test_data['image'] = test_data.apply(lambda row: str(row['id']) + ".jpg", axis=1)
test_data = test_data.sample(frac=1).reset_index(drop=True)


b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [3]:
# training data examples
training_data.head(10)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,12424,Women,Footwear,Shoes,Casual Shoes,Grey,Fall,2011.0,Casual,Timberland Women Casual Grey Casual Shoes,12424.jpg
1,8759,Men,Apparel,Topwear,Shirts,Red,Fall,2011.0,Casual,Indigo Nation Men Checks Shirt Red Shirts,8759.jpg
2,56151,Women,Personal Care,Makeup,Kajal and Eyeliner,Black,Spring,2017.0,Casual,Colorbar Kohl Intense Blackened Eye Liner 001,56151.jpg
3,17059,Women,Accessories,Belts,Belts,White,Winter,2015.0,Casual,Lino Perros Women Stripes White Belt,17059.jpg
4,59044,Women,Footwear,Shoes,Heels,Black,Winter,2012.0,Casual,Carlton London Women Black Heels,59044.jpg
5,56165,Women,Personal Care,Makeup,Kajal and Eyeliner,Brown,Spring,2017.0,Casual,Colorbar I-Define Truffle Eye Pencil 003,56165.jpg
6,45158,Men,Accessories,Watches,Watches,Brown,Winter,2016.0,Casual,CASIO ENTICER Men Brown Dial Analogue Watch A510,45158.jpg
7,39651,Men,Footwear,Shoes,Sports Shoes,White,Summer,2012.0,Sports,Lotto Men White Sports Shoes,39651.jpg
8,29243,Men,Footwear,Shoes,Sports Shoes,White,Fall,2012.0,Sports,ADIDAS Men 22Yds Trainer2 White Sports Shoes,29243.jpg
9,3852,Girls,Apparel,Topwear,Tshirts,Pink,Summer,2011.0,Casual,Disney Kids Girl's Pink Daisy Kidswear,3852.jpg


In [4]:
# testing data example
test_data.head(10)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,42089,Men,Footwear,Shoes,Sports Shoes,White,Summer,2012,Sports,ADIDAS Men White Corona Sports Shoes,42089.jpg
1,9204,Men,Footwear,Shoes,Casual Shoes,Black,Summer,2011,Casual,Puma Men Future Cat Remix SF Black Casual Shoes,9204.jpg
2,18005,Men,Apparel,Bottomwear,Shorts,Black,Summer,2011,Sports,Puma Men Long Logo Black Bermuda,18005.jpg
3,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,30039.jpg
4,11188,Men,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Carrera Men Dial steel finish strap Silver Wat...,11188.jpg
5,55802,Women,Apparel,Loungewear and Nightwear,Night suits,White,Summer,2017,Casual,SDL by Sweet Dreams Women Green Printed Night ...,55802.jpg
6,29584,Men,Apparel,Topwear,Tshirts,Black,Summer,2012,Casual,Indigo Nation Men Printed Black T-shirt,29584.jpg
7,25947,Women,Accessories,Scarves,Scarves,Brown,Summer,2012,Casual,Femella Women Brown Scarf,25947.jpg
8,9694,Men,Apparel,Topwear,Tshirts,White,Fall,2011,Casual,Classic Polo Men Vulcan White Polo Tshirts,9694.jpg
9,17885,Men,Apparel,Innerwear,Innerwear Vests,Grey,Summer,2016,Casual,Levis Men Comfort Style Grey Innerwear Vest,17885.jpg


In [5]:
#create additional images for training, while splitting dataset into two datasets for training and validation data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

#process training dataset images
training_generator = train_datagen.flow_from_dataframe(
    dataframe=training_data,
    directory=TRAINING_DATASET_PATH + "images",
    x_col="image",
    y_col="masterCategory",
    target_size=(96,96),
    batch_size=32,
    subset="training"
)

#process validation dataset images
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=training_data,
    directory=TRAINING_DATASET_PATH + "images",
    x_col="image",
    y_col="masterCategory",
    target_size=(96,96),
    batch_size=32,
    subset="validation"
)

#process testing dataset images
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=TEST_DATASET_PATH + "images",
    x_col="image",
    y_col="masterCategory",
    target_size=(96,96),
    batch_size=32
)

#create classes (categories to be sorted into)
classes = len(training_generator.class_indices)

D:\Anaconda\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 35536 validated image filenames belonging to 7 classes.
Found 8883 validated image filenames belonging to 7 classes.
Found 128 validated image filenames belonging to 4 classes.
